In [11]:
def makeCourseFilterCondition(level, price, duration):
        condition = " where "

        if (level == "None"):
            levelCondition = ""
        else:
            condition += f" toUpper(c.level) = \"{level.upper()}\" and "

        if price == "Free":
            condition += f"c.price = 0"
        else:
            condition += f"c.price != 0"
        
        if (duration != "Unlimit"):
            condition += f" and c.duration < {int(duration) + 1}"


        return condition


print(makeCourseFilterCondition("Basic", "Free", "8"))


 where  toUpper(c.level) = "BASIC" and c.price = 0 and c.duration < 9


In [3]:
import pandas as pd
import ast

DEFAULT_PATH = "/Users/nguyenvanviet/Work/Courses/DataGen/RecommandationCourseWeb-Neo4j/MyPaperDataSet/ETLCode/GDWData/"
fileName = DEFAULT_PATH + "JobDataForRDW.csv"
df = pd.read_csv(fileName)

def parse_knowledge(knowledge_str):
    try:
        # Convert the string representation of the list to an actual list
        return ast.literal_eval(knowledge_str)
    except Exception as e:
        print(f"Error parsing knowledge column: {e}")
        return []
    

df['Knowledge'] = df['Knowledge'].apply(parse_knowledge)

exploded_df = df.explode('Knowledge')
exploded_df.to_csv(DEFAULT_PATH + 'jobDataForRDW2.csv', index=False)

In [6]:
filter = exploded_df[exploded_df["Organization"] == "Assetario"]
filter.head(10)

,ProgrammingLanguage,Framework,Knowledge,Tool,Platform,jobCount,Career,Location,Organization,Web,Date,Month,Year,ID


In [3]:
a = [1, 2, 3, 4]
b = [ 5, 6]

len(set(a) & set(b))

0

In [4]:
a = [1, 2, 3, 4]
a_string = str(a)
print(a_string)


[1, 2, 3, 4]


In [2]:
",".join(["eh"])

'eh'

In [3]:
a = [1, 2, 3, 4]
b = [ 5, 6]

for i, j in zip(a, b):
    print(i ,"- ", j)

1 -  5
2 -  6


In [5]:

def convertDict( userSkill, column):
    new_structure = {"BASIC": [], "INTERMEDIATE": [], "ADVANCED": []}

    # Iterating through the original dictionary
    for language, level in userSkill[column].items():
        # If the level is already a key in the new structure, append the language to its list
        new_structure[level.upper()].append(language)
        
    # Updating the original dictionary
    userSkill[column] = new_structure

    return userSkill[column]

def convertDictBack( dict, column):
    original_structure = {}
    for level, languages in dict[column].items():
        for language in languages:
            original_structure[language] = level

    return original_structure

# Assuming u is your refactored dictionary

u = {
    "ProgrammingLanguage": {
        "Java": "Basic",
        "Python": "Advanced",
        "C#": "Basic",
        "SQL": "Advanced"
    }
}
ur = convertDict(u, "ProgrammingLanguage")
print(ur)
print(convertDictBack(u, "ProgrammingLanguage"))


{'BASIC': ['Java', 'C#'], 'INTERMEDIATE': [], 'ADVANCED': ['Python', 'SQL']}
{'Java': 'BASIC', 'C#': 'BASIC', 'Python': 'ADVANCED', 'SQL': 'ADVANCED'}


In [6]:
def pl_to_neo4j_condition(pl):
    # Initialize an empty list to hold condition parts
    conditions = []

    # Iterate through the dictionary
    for level, languages in pl.items():
        # Skip levels with empty language lists
        if not languages:
            continue

        # Format the condition part for the current level and add it to the list
        condition = "(toUpper(pl.level) = '{}' and pl.programmingLanguage in {})".format(level.upper(), languages)
        conditions.append(condition)

    # Join all condition parts with ' or '
    final_condition = ' or '.join(conditions)

    # Wrap the entire condition in 'where' clause if it's not empty
    if final_condition:
        final_condition = 'where ' + final_condition

    return final_condition

# Your programming languages dictionary
pl = {'BASIC': ['Java', 'C#'], 'INTERMEDIATE': [], 'ADVANCED': ['Python', 'SQL']}

# Convert to Neo4j condition
neo4j_condition = pl_to_neo4j_condition(pl)
print(neo4j_condition)


where (toUpper(pl.level) = 'BASIC' and pl.programmingLanguage in ['Java', 'C#']) or (toUpper(pl.level) = 'ADVANCED' and pl.programmingLanguage in ['Python', 'SQL'])


In [ ]:
def toNeo4jCondition( competencyColumn):
        # Initialize an empty list to hold condition parts
        conditions = []

        # Iterate through the dictionary
        for level, languages in competencyColumn.items():
            # Skip levels with empty language lists
            if not languages:
                continue

            # Format the condition part for the current level and add it to the list
            condition = "(toUpper(pl.level) = '{}' and pl.programmingLanguage in {})".format(level.upper(), languages)
            conditions.append(condition)

        # Join all condition parts with ' or '
        final_condition = ' or '.join(conditions)

        # Wrap the entire condition in 'where' clause if it's not empty
        if final_condition:
            final_condition = 'where ' + final_condition

        return final_condition

def updateUser(Competency, username):
    query = f''' match(u:User)
    where toUpper(u.username) = \"{username.upper()}\"
    match(u)-[:Known_programmingLanguage]->(pl : ProgrammingLanguage)  {toNeo4jCondition(Competency["ProgrammingLanguage"])} 
    optional match(u)-[:Known_knowledge]->(kl: Knowledge)  {toNeo4jCondition(Competency["Knowledge"])}
    optional match(u)-[:Known_platform]->(pf: Platform)  {toNeo4jCondition(Competency["Platform"])}
    optional match(u)-[:Known_tool]->(tl: Tool)  {toNeo4jCondition(Competency["Tool"])}
    optional match(u)-[:Known_framework]->(fw: Framework)  {toNeo4jCondition(Competency["Framework"])}
    set u.name = FullName, u.email = Email, u.phone = Phone
    merge(u)-[:Known_knowledge]->(kl)
    merge(u)-[:Known_programmingLanguage]->(pl)
    merge(u)-[:Known_platform]->(pf)
    merge(u)-[:Known_tool]->(tl)
    merge(u)-[:Known_framework]->(fw)
    '''
    a = queryToDataFrame(query)
    return True


In [10]:
data = {'Knowledge': ['DATA PIPELINES', 'DATABASES', 'COMPUTER SCIENCE', 'SQL', 'DATA WAREHOUSE', 'DATA ANALYSIS', 'ALGORITHM', 'OOP', 'DATABASE', 'ANALYSIS'], 
        'Platform': ['MYSQL', 'CLOUD', 'AZURE'],
          'Tool': ['ORACLE DATABASE', 'MYSQL', 'GIT'],
            'ProgrammingLanguage': {'BASIC': ['C#', 'PYTHON', 'JAVA'], 'INTERMEDIATE': ['SQL'], 'ADVANCED': ['C++', 'PYTHON']}, 
            'Framework': {'BASIC': ['ASP.NET', 'SPRING', 'SPARK', 'PANDAS'], 'INTERMEDIATE': ['SPARK'], 'ADVANCED': ['PANDAS']}}

def removeDuplicatesBasedOnHierarchy(data):
        # Define the hierarchy order
    hierarchy = ['ADVANCED', 'INTERMEDIATE', 'BASIC']
    
    for key in data.keys():
        if (key not in ["ProgrammingLanguage", "Framework"]): continue
        # Temporarily store items in a set for comparison, to avoid duplicates
        seen = set()
        
        # Iterate according to the hierarchy (from highest to lowest)
        for level in hierarchy:
            current_level_items = data[key][level]
            
            # If the current level items are not in a list form (e.g., empty string), skip
            if not isinstance(current_level_items, list):
                continue
            
            # Add current level items to the seen set (since they have the highest precedence)
            new_items = []
            for item in current_level_items:
                if item not in seen:
                    seen.add(item)
                    new_items.append(item)
                    
            # Update the current level with items not previously seen in higher levels
            data[key][level] = new_items
            
            # For lower levels, remove any items that have been seen in higher levels
            for lower_level in hierarchy[hierarchy.index(level)+1:]:
                lower_level_items = data[key][lower_level]
                # Skip if not in list form
                if not isinstance(lower_level_items, list):
                    continue
                data[key][lower_level] = [item for item in lower_level_items if item not in seen]


removeDuplicatesBasedOnHierarchy(data)
print(data)


{'Knowledge': ['DATA PIPELINES', 'DATABASES', 'COMPUTER SCIENCE', 'SQL', 'DATA WAREHOUSE', 'DATA ANALYSIS', 'ALGORITHM', 'OOP', 'DATABASE', 'ANALYSIS'], 'Platform': ['MYSQL', 'CLOUD', 'AZURE'], 'Tool': ['ORACLE DATABASE', 'MYSQL', 'GIT'], 'ProgrammingLanguage': {'BASIC': ['C#', 'JAVA'], 'INTERMEDIATE': ['SQL'], 'ADVANCED': ['C++', 'PYTHON']}, 'Framework': {'BASIC': ['ASP.NET', 'SPRING'], 'INTERMEDIATE': ['SPARK'], 'ADVANCED': ['PANDAS']}}


In [13]:
listz =[1, 2, 3, 4, 5, 6, 7, 8, 9]
listz = listz[:5]
listz

[1, 2, 3]

In [14]:
print(",".join(['SCIKIT-LEARN', 'XGBOOST', 'LIGHTGBM', 'MATPLOTLIB', 'PANDAS']))

SCIKIT-LEARN,XGBOOST,LIGHTGBM,MATPLOTLIB,PANDAS


In [1]:
def list2String(listVal):
    if type(listVal) == str: return listVal
    if type(listVal) == list:
        return ",".join(listVal)
    
    return str(listVal)

In [3]:
print(list2String("ok"))
print(list2String(2))
print(list2String(["dasda", "adsdas", "asd"]))

ok
2
dasda,adsdas,asd
